## Sentiment Analysis Score

Predict the sentiment analysis label, using a deep learning model for each query/emotion inside the relations file

In [1]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

## Load Tokenizer

Import and load the tokenizer from a `.pickle` file

In [2]:
import pickle
from pathlib import Path

In [3]:
tokenizer_file = Path('../datasets/sentiment140/tokenizer.pickle').resolve()
with tokenizer_file.open('rb') as file:
    tokenizer = pickle.load(file)

## Load Model

Load the sentiment analysis model, using the saved weights

In [6]:
from tensorflow.keras.layers import Input, Embedding, GRU
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dense
from tensorflow.keras.models import Sequential

In [7]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
embedding_dim = 200
input_length = 100
gru_units = 128
gru_dropout = 0.1
recurrent_dropout = 0.1
dropout = 0.1

In [8]:
model = Sequential()
model.add(Embedding(
    input_dim=input_dim,
    output_dim=embedding_dim,
    input_shape=(input_length,)
))

model.add(Bidirectional(GRU(
    gru_units,
    return_sequences=True,
    dropout=gru_dropout,
    recurrent_dropout=recurrent_dropout
)))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(1, activation='sigmoid'))

W0719 09:56:43.758275 4692825536 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0719 09:56:43.802737 4692825536 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0719 09:56:43.809999 4692825536 deprecation.py:506] From /Users/rmohashi/miniconda3/envs/emodata/lib/python3.6/site-packages/tensorflow/python/ops/init_op

In [9]:
weights_path = Path('../models/sentiment_analysis/model_weights.h5').resolve()
model.load_weights(weights_path.as_posix())

## Load Query Relations

Load the relations between queries and emotions from a `.json` file

In [10]:
import json

In [12]:
relations_path = Path('../query_relations.json')
with relations_path.open('r') as file:
    relations = json.load(file)

## Predict polarity

Predict the polarity of the texts, using the sentiment analysis model

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nlp import preprocess
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rmohashi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
dataset_dir = Path('../datasets/tweepy').resolve()

In [15]:
data_dict = {}

query_dict = {
    'query': [],
    'mean': [],
    'max': [],
    'min': [],
    'std': [],
    'count': [],
    'emotion': []
}

dir_files = os.listdir(dataset_dir)

with tqdm(total=len(dir_files)) as t:
    for filename in dir_files:
        dataset = pd.read_csv(os.path.join(dataset_dir, filename))
        cleaned_texts = preprocess(dataset.text, quiet=True)

        query = re.findall(r'(#[^.]+|:.+:)', filename)[0]

        predict_sequences = [text.split() for text in cleaned_texts]
        list_tokenized_predict = tokenizer.texts_to_sequences(predict_sequences)
        x_predict = pad_sequences(list_tokenized_predict, maxlen=100)

        result = model.predict(x_predict)
        
        emotion = relations[query]
        query_dict['query'].append(query)
        query_dict['mean'].append(np.mean(result))
        query_dict['max'].append(np.amax(result))
        query_dict['min'].append(np.amin(result))
        query_dict['count'].append(len(dataset))
        query_dict['std'].append(np.std(result))
        query_dict['emotion'].append(emotion)

        if emotion in data_dict:
            data_dict[emotion] = np.concatenate([data_dict[emotion], result])
        else:
            data_dict[emotion] = result
        
        t.update()

100%|██████████| 19/19 [02:59<00:00, 12.95s/it]


## Print Results

Print the queries/emotions and the values

In [10]:
df = pd.DataFrame(data=query_dict)
for emotion in df.emotion.unique():
    display(df[df.emotion == emotion])

,query,mean,max,min,std,count,emotion
0,:anxious_face_with_sweat:,0.428562,0.983642,0.004371,0.274272,199,fear
6,#worried,0.205504,0.879476,0.004883,0.210547,196,fear


,query,mean,max,min,std,count,emotion
1,#sad,0.073413,0.873629,0.002289,0.127914,200,sadness
2,:crying_face:,0.438269,0.996975,0.005851,0.296389,197,sadness


,query,mean,max,min,std,count,emotion
3,:red_heart:,0.770384,0.996633,0.042774,0.225747,200,joy
7,#joy,0.832007,0.997057,0.208914,0.152068,191,joy


,query,mean,max,min,std,count,emotion
4,:face_with_symbols_on_mouth:,0.403210,0.997371,0.010545,0.261377,194,angry
5,#pissed,0.230712,0.912333,0.008014,0.180684,200,angry


In [11]:
emotion_dict = {
    'emotion': [],
    'mean': [],
    'max': [],
    'min': [],
    'std': [],
    'count': []
}

for emotion, result in data_dict.items():
    emotion_dict['emotion'].append(emotion)
    emotion_dict['mean'].append(np.mean(result))
    emotion_dict['max'].append(np.amax(result))
    emotion_dict['min'].append(np.amin(result))
    emotion_dict['std'].append(np.std(result))
    emotion_dict['count'].append(len(result))
    
emotion_df = pd.DataFrame(data=emotion_dict)
display(emotion_df)

,emotion,mean,max,min,std,count
0,fear,0.317880,0.983642,0.004371,0.268948,395
1,sadness,0.254463,0.996975,0.002289,0.291740,397
2,joy,0.800486,0.997057,0.042774,0.195736,391
3,angry,0.315648,0.997371,0.008014,0.240100,394
